# Despliegue de un dash en AWS

Veremos algunas consideraciones importantes para correr nuestros dashboard sobre la nube. 

En primer lugar necesitaremos clonar los siguientes repositorios:

[Dash Examples](https://github.com/plotly/dash-sample-apps)

[]